In [1]:
#tweet cleaning
#https://www.kaggle.com/ragnisah/text-data-cleaning-tweets-analysis
#https://towardsdatascience.com/basic-data-cleaning-engineering-session-twitter-sentiment-data-95e5bd2869ec

#Topic Modeilling
#https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0

#Cleaning
#Puncuation
#lower case
#stopwords
#abbreviations

# Importing modules
import pandas as pd
import os

# Load the regular expression library
import re

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk as nltk
nltk.download('stopwords')
nltk.download('wordnet')
import gensim
from gensim.utils import simple_preprocess
from gensim.models.wrappers import LdaMallet
from nltk.corpus import stopwords
import string
import re
%matplotlib inline
pd.set_option('display.max_colwidth', 100)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import pyLDAvis.gensim
import pickle 
import pyLDAvis
#import spacy
#from scispacy.abbreviation import AbbreviationDetector

#sentiment analysis
#https://medium.com/swlh/simple-sentiment-analysis-for-nlp-beginners-and-everyone-else-using-vader-and-textblob-728da3dbe33d
import sqlite3
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\megha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\megha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\megha\anaconda3\lib\site-packages\sklearn\decomposition\_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [2]:
warnings.filterwarnings("ignore",category=DeprecationWarning)

C:\Users\megha\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
diabetes2020 = pd.read_csv('diabetes2020_lda.csv')

In [4]:
# Print head
diabetes2020.head(10)

,Post ID,Author,Publish Date,Body,text_gens_nonstop,text_processed,text_tokenized,text_nlk_nonstop,text_nonstop_spacy,text_stemmed,text_lemmatized,Dominant,month_year
0,eixjcz,Temptazn,2020-01-02 04:16:00,"Hi all, \n\nSorry for the long post, need some guidance.\n\nJust before Christmas I injured my r...","Hi all, Sorry long post, need guidance. Just Christmas I injured right knee bad I can't walk. So...",hi all sorry long post need guidance just christmas i injured right knee bad i cant walk so dec ...,"['hi', 'all', 'sorry', 'long', 'post', 'need', 'guidance', 'just', 'christmas', 'i', 'injured', ...","['sorry', 'long', 'post', 'need', 'guidance', 'christmas', 'injured', 'right', 'knee', 'bad', 'c...","['sorry', 'long', 'post', 'need', 'guidance', 'christmas', 'injured', 'right', 'knee', 'bad', 'c...","['sorri', 'long', 'post', 'need', 'guidanc', 'christma', 'injur', 'right', 'knee', 'bad', 'cant'...","['sorry', 'long', 'post', 'need', 'guidance', 'christmas', 'injured', 'right', 'knee', 'bad', 'c...",Topic 8,2020-01
1,fcu0bkz,alanstrainor,2020-01-02 04:27:00,"I am not a doctor, and I have never even heard of the medication you're taking for your knee.\n\...","I doctor, I heard medication you're taking knee. But sugars huge. How long T2? Is recent diagnos...",i doctor i heard medication youre taking knee but sugars huge how long t is recent diagnosis is ...,"['i', 'doctor', 'i', 'heard', 'medication', 'youre', 'taking', 'knee', 'but', 'sugars', 'huge', ...","['doctor', 'heard', 'medication', 'taking', 'knee', 'sugars', 'huge', 'long', 'recent', 'diagnos...","['doctor', 'heard', 'medication', 'taking', 'knee', 'sugars', 'huge', 'long', 'recent', 'diagnos...","['doctor', 'heard', 'medic', 'take', 'knee', 'sugar', 'huge', 'long', 'recent', 'diagnosi', 'cha...","['doctor', 'heard', 'medication', 'taking', 'knee', 'sugar', 'huge', 'long', 'recent', 'diagnosi...",Topic 10,2020-01
2,fcu0i9t,thot_sauce,2020-01-02 04:31:00,Steroids spike blood sugar. Like a lot. Did the doctor not know about your diabetes? When I got ...,"Steroids spike blood sugar. Like lot. Did doctor know diabetes? When I got wisdom teeth removed,...",steroids spike blood sugar like lot did doctor know diabetes when i got wisdom teeth removed sur...,"['steroids', 'spike', 'blood', 'sugar', 'like', 'lot', 'did', 'doctor', 'know', 'diabetes', 'whe...","['steroids', 'spike', 'blood', 'sugar', 'lot', 'doctor', 'know', 'diabetes', 'got', 'wisdom', 't...","['steroids', 'spike', 'blood', 'sugar', 'lot', 'doctor', 'know', 'diabetes', 'got', 'wisdom', 't...","['steroid', 'spike', 'blood', 'sugar', 'lot', 'doctor', 'know', 'diabet', 'got', 'wisdom', 'teet...","['steroid', 'spike', 'blood', 'sugar', 'lot', 'doctor', 'know', 'diabetes', 'got', 'wisdom', 'te...",Topic 7,2020-01
3,fcu0pas,DaPoole420,2020-01-02 04:35:00,Type 1 all I can say those numbers are crazy high. I know some meds raise bs but if your still ...,Type 1 I numbers crazy high. I know meds raise bs high stopping .. id suggest seeing primary,type i numbers crazy high i know meds raise bs high stopping id suggest seeing primary,"['type', 'i', 'numbers', 'crazy', 'high', 'i', 'know', 'meds', 'raise', 'bs', 'high', 'stopping'...","['type', 'numbers', 'crazy', 'high', 'know', 'meds', 'raise', 'bs', 'high', 'stopping', 'suggest...","['type', 'numbers', 'crazy', 'high', 'know', 'meds', 'raise', 'bs', 'high', 'stopping', 'suggest...","['type', 'number', 'crazi', 'high', 'know', 'med', 'rais', 'bs', 'high', 'stop', 'suggest', 'see...","['type', 'number', 'crazy', 'high', 'know', 'med', 'raise', 'b', 'high', 'stopping', 'suggest', ...",Topic 7,2020-01
4,fcu0q11,thot_sauce,2020-01-02 04:35:00,Steroids spike blood sugar. Like a lot. Did the doctor not know about your diabetes? That's odd ...,Steroids spike blood sugar. Like lot. Did doctor know diabetes? That's odd didn't prescribed you...,steroids spike blood sugar like lot did doctor know diabetes thats odd didnt prescribed you when...,"['steroids', 'spike', 'blood', 'su

In [5]:
list(diabetes2020.columns)

['Post ID',
 'Author',
 'Publish Date',
 'Body',
 'text_gens_nonstop',
 'text_processed',
 'text_tokenized',
 'text_nlk_nonstop',
 'text_nonstop_spacy',
 'text_stemmed',
 'text_lemmatized',
 'Dominant',
 'month_year']

In [6]:
#drop unnecessary columns
diabetes2020 = diabetes2020.drop(columns = ['text_gens_nonstop', 'text_processed', 'text_tokenized', 'text_nlk_nonstop', 'text_nonstop_spacy', 'text_stemmed', 'text_lemmatized'])
diabetes2020

,Post ID,Author,Publish Date,Body,Dominant,month_year
0,eixjcz,Temptazn,2020-01-02 04:16:00,"Hi all, \n\nSorry for the long post, need some guidance.\n\nJust before Christmas I injured my r...",Topic 8,2020-01
1,fcu0bkz,alanstrainor,2020-01-02 04:27:00,"I am not a doctor, and I have never even heard of the medication you're taking for your knee.\n\...",Topic 10,2020-01
2,fcu0i9t,thot_sauce,2020-01-02 04:31:00,Steroids spike blood sugar. Like a lot. Did the doctor not know about your diabetes? When I got ...,Topic 7,2020-01
3,fcu0pas,DaPoole420,2020-01-02 04:35:00,Type 1 all I can say those numbers are crazy high. I know some meds raise bs but if your still ...,Topic 7,2020-01
4,fcu0q11,thot_sauce,2020-01-02 04:35:00,Steroids spike blood sugar. Like a lot. Did the doctor not know about your diabetes? That's odd ...,Topic 6,2020-01
...,...,...,...,...,...,...
55708,goxee5q,satxlonghorn1,2021-02-26 18:26:00,"I didn't have any side effects, just fyi. Some people don't.",Topic 8,2021-02
55709,goxfko5,AccidentalDragon,2021-02-26 18:36:00,Fasting brought my numbers was down from where they were! I do keto omad and came down from 160-...,Topic 7,2021-02
55710,lthxwe,Wackydetective,2021-02-26 22:17:00,I've been battling depression and anxiety like so many of us are living in end times. I was so a...,Topic 2,2021-02
55711,lu13pm,Nervous_Finger1486,2021-02-27 16:27:00,So it’s been a week since I was diagnosed. Metformin 1000mg twice a day. BS has been below 200 l...,Topic 8,2021-02


In [7]:
types = diabetes2020.dtypes
types

Post ID         object
Author          object
Publish Date    object
Body            object
Dominant        object
month_year      object
dtype: object

In [8]:
#convert object to string
diabetes2020['Body'] = diabetes2020['Body'].astype(str)

In [9]:
#sentiment analysis with textblob
#https://medium.com/swlh/simple-sentiment-analysis-for-nlp-beginners-and-everyone-else-using-vader-and-textblob-728da3dbe33d

#load the descriptions into textblob
desc_blob = [TextBlob(desc) for desc in diabetes2020['Body']]

#add the sentiment metrics to the dataframe
diabetes2020['tb_Pol'] = [b.sentiment.polarity for b in desc_blob]
diabetes2020['tb_Subj'] = [b.sentiment.subjectivity for b in desc_blob]

diabetes2020.head()

,Post ID,Author,Publish Date,Body,Dominant,month_year,tb_Pol,tb_Subj
0,eixjcz,Temptazn,2020-01-02 04:16:00,"Hi all, \n\nSorry for the long post, need some guidance.\n\nJust before Christmas I injured my r...",Topic 8,2020-01,-0.216131,0.478132
1,fcu0bkz,alanstrainor,2020-01-02 04:27:00,"I am not a doctor, and I have never even heard of the medication you're taking for your knee.\n\...",Topic 10,2020-01,-0.008750,0.486250
2,fcu0i9t,thot_sauce,2020-01-02 04:31:00,Steroids spike blood sugar. Like a lot. Did the doctor not know about your diabetes? When I got ...,Topic 7,2020-01,-0.291111,0.622222
3,fcu0pas,DaPoole420,2020-01-02 04:35:00,Type 1 all I can say those numbers are crazy high. I know some meds raise bs but if your still ...,Topic 7,2020-01,0.030000,0.620000
4,fcu0q11,thot_sauce,2020-01-02 04:35:00,Steroids spike blood sugar. Like a lot. Did the doctor not know about your diabetes? That's odd ...,Topic 6,2020-01,-0.052381,0.426068


In [10]:
#load VADER
analyzer = SentimentIntensityAnalyzer()

#Add VADER metrics to dataframe
diabetes2020['compound'] = [analyzer.polarity_scores(v)['compound'] for v in diabetes2020['Body']]
diabetes2020['neg'] = [analyzer.polarity_scores(v)['neg'] for v in diabetes2020['Body']]
diabetes2020['neu'] = [analyzer.polarity_scores(v)['neu'] for v in diabetes2020['Body']]
diabetes2020['pos'] = [analyzer.polarity_scores(v)['pos'] for v in diabetes2020['Body']]

diabetes2020.tail(50)

,Post ID,Author,Publish Date,Body,Dominant,month_year,tb_Pol,tb_Subj,compound,neg,neu,pos
55663,lsfcp6,RiChessReadit,2021-02-25 11:58:00,"So I've been eating pretty much the same lunch for the last few weeks, a low carb wheat bread ha...",Topic 1,2021-02,0.030000,0.365283,-0.5956,0.082,0.878,0.040
55664,gor0kig,modernjess,2021-02-25 12:39:00,"IME, it's not uncommon to get occasional errant readings. Maybe do a second reading? In any case...",Topic 7,2021-02,0.170000,0.625000,0.7845,0.000,0.769,0.231
55665,gor0ydz,RiChessReadit,2021-02-25 12:41:00,"I checked twice with different fingers, first reading was 110 the other was 109. The higher read...",Topic 7,2021-02,0.184375,0.519792,0.2516,0.067,0.817,0.116
55666,gorgvns,alan_s,2021-02-25 14:33:00,"&gt; whole lunch is around 40-50g net carbs. \n\nTomorrow check every 30 minutes (30, 60, 90) af...",Topic 6,2021-02,-0.078000,0.401333,0.0000,0.000,1.000,0.000
55667,lsrhm7,MollyMooSkadoo,2021-02-25 22:12:00,I just need to rant. This might not be coherent.\n\nI was diagnosed this week. I was put on metf...,Topic 9,2021-02,0.018177,0.528664,-0.9924,0.152,0.767,0.081
55668,gosxf7x,BeerLaoDrinker,2021-02-25 22:29:00,&amp;#x200B;\n\n&gt;Why not just take the medicine and hope it works well enough for a while?\n\...,Topic 5,2021-02,0.124150,0.683673,0.8738,0.033,0.852,0.115
55669,gosxsmn,MollyMooSkadoo,2021-02-25 22:34:00,"I wish. I'm unfortunately infertile. I have diabetes because I have PCOS, and didn't know that p...",Topic 6,2021-02,-0.400000,0.700000,-0.7003,0.188,0.752,0.060
55670,got1jpp,hidflect1,2021-02-25 23:26:00,"Ironically, the diabetes diet can be way cheaper or way more expensive than a regular one. Steak...",Topic 1,2021-02,0.027941,0.543577,-0.7050,0.105,0.819,0.076
55671,got7dlj,BeerLaoDrinker,2021-02-26 00:53:00,"Sadly, you will have to make some lifestyle changes. You are fairly young, so probably not much ...",Topic 3,2021-02,0.118214,0.646964,0.4054,0.021,0.916,0.063
55672,gotam8k,xoxoahooves,2021-02-26 01:43:00,I was diagnosed in January and I just went to a diabetes educator on Monday. My doctor recommend...,Topic 4,2021-02,0.116176,0.472549,0.7460,0.000,0.928,0.072


In [11]:
#drop textblob columns
diabetes2020 = diabetes2020.drop(columns = ['tb_Pol', 'tb_Subj'])
diabetes2020

,Post ID,Author,Publish Date,Body,Dominant,month_year,compound,neg,neu,pos
0,eixjcz,Temptazn,2020-01-02 04:16:00,"Hi all, \n\nSorry for the long post, need some guidance.\n\nJust before Christmas I injured my r...",Topic 8,2020-01,-0.8693,0.101,0.828,0.071
1,fcu0bkz,alanstrainor,2020-01-02 04:27:00,"I am not a doctor, and I have never even heard of the medication you're taking for your knee.\n\...",Topic 10,2020-01,-0.3892,0.099,0.810,0.091
2,fcu0i9t,thot_sauce,2020-01-02 04:31:00,Steroids spike blood sugar. Like a lot. Did the doctor not know about your diabetes? When I got ...,Topic 7,2020-01,-0.5423,0.114,0.807,0.080
3,fcu0pas,DaPoole420,2020-01-02 04:35:00,Type 1 all I can say those numbers are crazy high. I know some meds raise bs but if your still ...,Topic 7,2020-01,-0.3818,0.114,0.886,0.000
4,fcu0q11,thot_sauce,2020-01-02 04:35:00,Steroids spike blood sugar. Like a lot. Did the doctor not know about your diabetes? That's odd ...,Topic 6,2020-01,0.2023,0.073,0.839,0.088
...,...,...,...,...,...,...,...,...,...,...
55708,goxee5q,satxlonghorn1,2021-02-26 18:26:00,"I didn't have any side effects, just fyi. Some people don't.",Topic 8,2021-02,0.2023,0.000,0.847,0.153
55709,goxfko5,AccidentalDragon,2021-02-26 18:36:00,Fasting brought my numbers was down from where they were! I do keto omad and came down from 160-...,Topic 7,2021-02,0.0000,0.000,1.000,0.000
55710,lthxwe,Wackydetective,2021-02-26 22:17:00,I've been battling depression and anxiety like so many of us are living in end times. I was so a...,Topic 2,2021-02,0.9092,0.083,0.724,0.192
55711,lu13pm,Nervous_Finger1486,2021-02-27 16:27:00,So it’s been a week since I was diagnosed. Metformin 1000mg twice a day. BS has been below 200 l...,Topic 8,2021-02,-0.7163,0.120,0.805,0.075


In [12]:
#download data with topics and sentiment score
diabetes2020.to_csv('diabetes2020_sentiment.csv', index = False)

In [13]:
#group average sentiment by topic
average_sentiment = diabetes2020.groupby('Dominant')['compound', 'neg', 'neu', 'pos'].mean()
average_sentiment

<ipython-input-13-7236e06eea2d>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  average_sentiment = diabetes2020.groupby('Dominant')['compound', 'neg', 'neu', 'pos'].mean()


,compound,neg,neu,pos
Dominant,,,,
Topic 1,0.300836,0.048392,0.813433,0.137854
Topic 10,0.113141,0.065702,0.844455,0.089845
Topic 2,0.155494,0.069624,0.821434,0.108944
Topic 3,0.439600,0.055861,0.678710,0.265431
Topic 4,0.186909,0.072087,0.818216,0.109687
Topic 5,0.306229,0.057371,0.780556,0.162077
Topic 6,0.241408,0.042647,0.853895,0.103456
Topic 7,0.131479,0.055915,0.860303,0.083784
Topic 8,0.149889,0.066940,0.829295,0.103758


In [14]:
average_sentiment.to_csv('diabetes2020_sentiment_by_topic.csv', index = False)

In [16]:
#group by month 
average_sentiment_month = diabetes2020.groupby('month_year')['compound', 'neg', 'neu', 'pos'].mean()
average_sentiment_month

<ipython-input-16-02b571f7f255>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  average_sentiment_month = diabetes2020.groupby('month_year')['compound', 'neg', 'neu', 'pos'].mean()


,compound,neg,neu,pos
month_year,,,,
2020-01,0.226141,0.063544,0.797611,0.138849
2020-02,0.217639,0.061124,0.807952,0.130917
2020-03,0.184246,0.066209,0.801474,0.132323
2020-04,0.211589,0.061685,0.806701,0.131613
2020-05,0.234781,0.058353,0.804155,0.136909
2020-06,0.233449,0.059627,0.804245,0.136131
2020-07,0.241477,0.062626,0.799429,0.137950
2020-08,0.253727,0.057288,0.806824,0.135892
2020-09,0.249394,0.058229,0.805030,0.136739


In [17]:
average_sentiment_month.to_csv('diabetes2020_sentiment_by_month.csv', index = False)

In [18]:
#sort by authors & dates
diabetes_authors = diabetes2020.sort_values(by=['Author', 'Publish Date'])
diabetes_authors

,Post ID,Author,Publish Date,Body,Dominant,month_year,compound,neg,neu,pos
3451,fop3oat,#NAME?,2020-04-26 18:52:00,I felt that a little too hard...\n\nI hate it too.,Topic 2,2020-04,-0.5888,0.349,0.651,0.000
3640,fp48x5b,#NAME?,2020-04-30 17:47:00,"Look on the bright side if you can, now is the best time in history to get diabetes. If you take...",Topic 9,2020-04,0.9151,0.000,0.686,0.314
10864,j1gpb0,#NAME?,2020-09-28 10:40:00,So My dad (50) was diagnosed with diabetes few days ago. Readings were 180 early in the morning ...,Topic 4,2020-09,-0.2719,0.084,0.841,0.075
38453,g9htkfl,#NAME?,2020-10-20 17:47:00,Tell me about it. I'm like 112 pounds and I have diabetes. I've never been overweight. I'm young...,Topic 2,2020-10,0.3915,0.064,0.785,0.151
38752,g9p3zz9,#NAME?,2020-10-22 15:15:00,Yeah I don't randomly tell everyone. Just people close to me in case anything ever happens to me...,Topic 7,2020-10,0.6249,0.000,0.873,0.127
...,...,...,...,...,...,...,...,...,...,...
35263,g4doju7,zymurgy_bc,2020-09-07 14:22:00,I can't handle Metformin. Two days in and I'm blowing diarrhea at the most inconvenient of times...,Topic 9,2020-09,0.2280,0.131,0.751,0.118
35266,g4e4407,zymurgy_bc,2020-09-07 16:55:00,"Well, I think it's definitely ""your mileage may vary"" and no one really says otherwise. Glycemi...",Topic 1,2020-09,0.9639,0.101,0.741,0.158
35269,g4ebzvy,zymurgy_bc,2020-09-07 18:09:00,I think he's less than 6 months in... no idea how much education has been taken in yet. My doct...,Topic 5,2020-09,-0.6640,0.075,0.884,0.041
35894,g54ypi0,zymurgy_bc,2020-09-13 10:50:00,"Not according to Trump... and he's a stable genius, unlike them fact checking fake news sources ...",Topic 5,2020-09,0.4294,0.119,0.653,0.228


In [19]:
diabetes_authors.to_csv('diabetes2020_authors_sentiment.csv', index = False)